# Further Pre-training MobileBERT MLM with Non-Federated, Centralized Training (Shakespeare)

In [1]:
# Copyright 2020, The TensorFlow Federated Authors.
# Copyright 2020, Ronald Seoh
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Google Colab settings

In [2]:
# Use Google Colab
use_colab = True

# Is this notebook running on Colab?
# If so, then google.colab package (github.com/googlecolab/colabtools)
# should be available in this environment

# Previous version used importlib, but we could do the same thing with
# just attempting to import google.colab
try:
    from google.colab import drive
    colab_available = True
except:
    colab_available = False

if use_colab and colab_available:
    # Mount Google Drive root directory
    drive.mount('/content/drive')

    # cd to the appropriate working directory under my Google Drive
    %cd '/content/drive/My Drive/Colab Notebooks/BERTerated'
    
    # List the directory contents
    !ls

# IPython reloading magic
%load_ext autoreload
%autoreload 2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/BERTerated
datasets
fedavg_client.py
fedavg.py
huggingface_keras_layers.py
LICENSE
mobilebert_mlm_shakespeare_centralized.ipynb
mobilebert_mlm_shakespeare_fedavg.ipynb
mobilebert_mlm_stackoverflow_fedavg.ipynb
__pycache__
README.md
requirements.txt
tff_cache
transformers_cache
utils.py


In [3]:
# Install required packages
!pip install -r requirements.txt

## Import packages

In [4]:
import os
import sys
import random

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow_text as tf_text
import transformers

import nest_asyncio
nest_asyncio.apply()

import fedavg
import fedavg_client
import datasets
import utils

# Random seed settings
random_seed = 692
random.seed(random_seed) # Python
np.random.seed(random_seed) # NumPy
tf.random.set_seed(random_seed) # TensorFlow

# Tensorflow GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Test the TFF is working:
tff.federated_computation(lambda: 'Hello, World!')()

Num GPUs Available:  1


b'Hello, World!'

In [5]:
# Print version information
print("Python version: " + sys.version)
print("NumPy version: " + np.__version__)
print("TensorFlow version: " + tf.__version__)
print("TensorFlow Federated version: " + tff.__version__)
print("Transformers version: " + transformers.__version__)

Python version: 3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0]
NumPy version: 1.18.5
TensorFlow version: 2.3.0
TensorFlow Federated version: 0.17.0
Transformers version: 3.4.0


In [ ]:
!nvidia-smi

## Experiment Settings

In [6]:
TOTAL_ROUNDS = 2 # Number of total training rounds
ROUNDS_PER_EVAL = 1 # How often to evaluate

TRAIN_CLIENTS_PER_ROUND = 3 # How many clients to sample per round.
TEST_CLIENTS_PER_ROUND = 3 # How many clients to sample per round for test data

# Number of epochs in the client to take per round.
CLIENT_EPOCHS_PER_ROUND = 3

# Epochs to train in centralized setting
CENTRALIZED_EPOCHS = CLIENT_EPOCHS_PER_ROUND * TOTAL_ROUNDS

BATCH_SIZE = 8 # Batch size used on the client.
TEST_BATCH_SIZE = 8 # Minibatch size of test data.

# Maximum length of input token sequence for BERT.
BERT_MAX_SEQ_LENGTH = 128

# Optimizer configuration
SERVER_LEARNING_RATE = 1.0 # Server learning rate.
CLIENT_LEARNING_RATE = 2e-5 # Client learning rate

CENTRALIZED_LEARNING_RATE = SERVER_LEARNING_RATE * CLIENT_LEARNING_RATE

## Dataset

### Load the Shakespeare dataset

In [7]:
train_client_data, test_client_data = tff.simulation.datasets.shakespeare.load_data(cache_dir='./tff_cache')

### Tokenizer

In [8]:
mobilebert_tokenizer = transformers.MobileBertTokenizer.from_pretrained(
    'google/mobilebert-uncased', cache_dir='./transformers_cache')

In [9]:
# Imitate transformers tokenizer with TF.Text Tokenizer
tokenizer_tf_text, vocab_lookup_table, special_ids_mask_table = datasets.preprocessing_for_bert.convert_huggingface_tokenizer(mobilebert_tokenizer)

In [10]:
# Test if our new tokenizer works
ttt = tokenizer_tf_text.tokenize("This is a test.")
print("TF Text tokenizer output shape:", tf.shape(ttt.to_tensor()))
print(tf.squeeze(ttt.to_tensor(), axis=-1))
mobilebert_tokenizer.decode(tf.squeeze(ttt, axis=-1).to_list()[0])

TF Text tokenizer output shape: tf.Tensor([1 5 1], shape=(3,), dtype=int32)
tf.Tensor([[2023 2003 1037 3231 1012]], shape=(1, 5), dtype=int32)


'this is a test.'

### Preprocessing

In [11]:
def check_empty_snippet(x):
    return tf.strings.length(x['snippets']) > 0

def tokenizer_and_mask_wrapped(x):

    masked, labels = datasets.preprocessing_for_bert.tokenize_and_mask(tf.reshape(x['snippets'], shape=[1]),
                                                                       max_seq_length=BERT_MAX_SEQ_LENGTH,
                                                                       bert_tokenizer_tf_text=tokenizer_tf_text,
                                                                       vocab_lookup_table=vocab_lookup_table,
                                                                       special_ids_mask_table=special_ids_mask_table,
                                                                       cls_token_id=mobilebert_tokenizer.cls_token_id,
                                                                       sep_token_id=mobilebert_tokenizer.sep_token_id,
                                                                       pad_token_id=mobilebert_tokenizer.pad_token_id,
                                                                       mask_token_id=mobilebert_tokenizer.mask_token_id)

    return (masked, labels)

def preprocess_for_train(train_dataset):
    return (
        train_dataset
        # Tokenize each samples using MobileBERT tokenizer
        .map(tokenizer_and_mask_wrapped, num_parallel_calls=tf.data.experimental.AUTOTUNE, deterministic=False)
        # Shuffle
        .shuffle(100000)
        # Form minibatches
        # Use drop_remainder=True to force the batch size to be exactly BATCH_SIZE
        # and make the shape **exactly** (BATCH_SIZE, SEQ_LENGTH)
        .batch(BATCH_SIZE, drop_remainder=True)
        # Repeat to make each client train multiple epochs
        .repeat(count=CENTRALIZED_EPOCHS)
    )
    
def preprocess_for_test(test_dataset):
    return (
        test_dataset
        # Tokenize each samples using MobileBERT tokenizer
        .map(tokenizer_and_mask_wrapped, num_parallel_calls=tf.data.experimental.AUTOTUNE, deterministic=False)
        # Shuffle
        .shuffle(100000)
        # Form minibatches
        # Use drop_remainder=True to force the batch size to be exactly TEST_BATCH_SIZE
        # and make the shape **exactly** (TEST_BATCH_SIZE, SEQ_LENGTH)
        .batch(TEST_BATCH_SIZE, drop_remainder=True)
    )

In [12]:
train_client_data_all_merged = train_client_data.create_tf_dataset_for_client(
    train_client_data.client_ids[0]).filter(check_empty_snippet)

if len(train_client_data.client_ids) > 1:
    for i in range(1, len(train_client_data.client_ids)):
        train_client_data_all_merged = train_client_data_all_merged.concatenate(
            train_client_data.create_tf_dataset_for_client(train_client_data.client_ids[i]).filter(check_empty_snippet))

In [13]:
train_client_data_all_merged = preprocess_for_train(train_client_data_all_merged)

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [14]:
train_client_data_all_merged.element_spec

(TensorSpec(shape=(8, 128), dtype=tf.int32, name=None),
 TensorSpec(shape=(8, 128), dtype=tf.int32, name=None))

In [15]:
test_client_data_all_merged = test_client_data.create_tf_dataset_for_client(
    test_client_data.client_ids[0]).filter(check_empty_snippet)

if len(test_client_data.client_ids) > 1:
    for i in range(1, len(test_client_data.client_ids)):
        test_client_data_all_merged = test_client_data_all_merged.concatenate(
            test_client_data.create_tf_dataset_for_client(test_client_data.client_ids[i]).filter(check_empty_snippet))

In [16]:
test_client_data_all_merged = preprocess_for_test(test_client_data_all_merged)

In [17]:
test_input = list(test_client_data_all_merged.take(1).as_numpy_iterator())

In [18]:
print(test_input)

[(array([[  101,  1996, 13410, ...,     0,     0,     0],
       [  101,  1031,   103, ...,     0,     0,     0],
       [  101,  2204, 25287, ...,     0,     0,     0],
       ...,
       [  101, 24766,   103, ...,     0,     0,     0],
       [  101,  2026,  2935, ...,     0,     0,     0],
       [  101, 11127,  1005, ...,     0,     0,     0]], dtype=int32), array([[ -100,  -100,  -100, ...,  -100,  -100,  -100],
       [ -100,  -100,  9466, ...,  -100,  -100,  -100],
       [ -100,  -100,  -100, ...,  -100,  -100,  -100],
       ...,
       [ -100, 21658,  1010, ...,  -100,  -100,  -100],
       [ -100,  -100,  -100, ...,  -100,  -100,  -100],
       [ -100,  -100,  -100, ...,  -100,  -100,  -100]], dtype=int32))]


## Model

In [19]:
mobilebert_model = transformers.TFMobileBertForPreTraining.from_pretrained(
    'google/mobilebert-uncased', cache_dir='./transformers_cache')

All model checkpoint layers were used when initializing TFMobileBertForPreTraining.

All the layers of TFMobileBertForPreTraining were initialized from the model checkpoint at google/mobilebert-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMobileBertForPreTraining for predictions without further training.


In [20]:
print(mobilebert_model.config)

MobileBertConfig {
  "_name_or_path": "google/mobilebert-uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_activation": false,
  "embedding_size": 128,
  "hidden_act": "relu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "intra_bottleneck_size": 128,
  "key_query_shared_bottleneck": true,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "mobilebert",
  "normalization_type": "no_norm",
  "num_attention_heads": 4,
  "num_feedforward_networks": 4,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "trigram_input": true,
  "true_hidden_size": 128,
  "type_vocab_size": 2,
  "use_bottleneck": true,
  "use_bottleneck_attention": false,
  "vocab_size": 30522
}



In [21]:
# Due to the limitations with Keras subclasses, we can only use the main layer part from pretrained models
# and add output heads by ourselves
mobilebert_keras_converted = utils.convert_huggingface_mlm_to_keras(
    huggingface_model=mobilebert_model,
    max_seq_length=BERT_MAX_SEQ_LENGTH,
    batch_size=BATCH_SIZE)

In [22]:
mobilebert_keras_converted.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
mobilebert (TFMobileBertMain ((None, 128, 512), (None, 24581888  
_________________________________________________________________
standalone_tf_mobile_bert_ml (None, 128, 30522)        15921466  
Total params: 40,503,354
Trainable params: 40,503,354
Non-trainable params: 0
_________________________________________________________________


## Training

### Training setups

In [23]:
mobilebert_keras_converted.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=utils.MaskedLMCrossEntropy(),
)

In [24]:
mobilebert_keras_converted.fit(
    train_client_data_all_merged,
    epochs=CENTRALIZED_EPOCHS,
)

Epoch 1/6
11934/11934 [==============================] - 1653s 138ms/step - loss: 3.4297
Epoch 2/6
11934/11934 [==============================] - 1676s 140ms/step - loss: 2.9899
Epoch 3/6
11934/11934 [==============================] - 1677s 141ms/step - loss: 2.7028
Epoch 4/6
11934/11934 [==============================] - 1674s 140ms/step - loss: 2.4852
Epoch 5/6
11934/11934 [==============================] - 1674s 140ms/step - loss: 2.2713
Epoch 6/6
11934/11934 [==============================] - 1679s 141ms/step - loss: 2.0708


In [25]:
mobilebert_keras_converted.evaluate(
    test_client_data_all_merged,
)

288/288 [==============================] - 11s 37ms/step - loss: 3.3100


3.3099985122680664

In [26]:
mobilebert_keras_converted.save('./results/mobilebert_mlm_shakespeare_centralized/20201121_01/model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: ./results/mobilebert_mlm_shakespeare_centralized/20201121_01/model/assets


INFO:tensorflow:Assets written to: ./results/mobilebert_mlm_shakespeare_centralized/20201121_01/model/assets
